# This notebook evaluates the predictions of the TinyBert Baseline output

In [1]:
import pandas as pd
from sklearn.metrics import f1_score
import os

In [2]:
#get path information
product_path = '../../../../src/data/product'
train_test_all_filtered_path = os.path.join(product_path, 'train_test_split/output_unfiltered_tables/large/after_manual_checking/baselines')

In [3]:
#load the TinyBert results
res = pd.read_csv('TinyBert_Results/predict_results_None.txt', sep='\t')
res.drop('index', axis=1, inplace=True)

In [4]:
#load the test set
real = pd.read_csv(os.path.join(train_test_all_filtered_path,'df_test.csv'))

In [16]:
#tinybert score
f1_score(real.label, res.prediction, average='micro')

0.8329422806194275

In [6]:
real

,sentence1,label
0,WD 2TB Elements Portable External Hard Drive -...,442
1,GoPro Head Strap + Quick Clip nan,591
2,GoPro 3 way grip Arm GoPro nan,245
3,Akérat Smoothing Exfoliating Cream <p>Targeted...,692
4,Moisturizing Self-Tanning Silky Gel <p>Non-gre...,223
...,...,...
10650,Apple Lightning to USB-C Cable (2m) Apple Ligh...,70
10651,Apple Lightning to VGA adapter Apple Lightning...,184
10652,Apple Lightning to Digital AV adapter Apple Li...,45
10653,Rolex Watches Oyster Perpetual GMT-Master II 1...,565


In [31]:
real.label.nunique()

1410

In [15]:
res.prediction

0        442
1        591
2        245
3        570
4        569
        ... 
10650     70
10651    184
10652    648
10653    565
10654     54
Name: prediction, Length: 10655, dtype: int64

In [10]:
final = pd.concat([real,res], axis=1)

In [13]:
final.groupby('label')

In [35]:
f1_scores = f1_score(real.label, res.prediction, average=None, labels=real.label)
f1_scores_with_labels = {label:score for label,score in zip(real.label, f1_scores)}

In [43]:
df_f1 = pd.DataFrame.from_dict(f1_scores_with_labels, orient="index").reset_index().rename(columns={'index':'label',0:'f1'}).sort_values(by=['label'])

In [54]:
df_join_sentence = pd.merge(df_f1, real.drop_duplicates(subset=['label']), how="left", on=["label", "label"])

In [55]:
df_join_sentence

,label,f1,sentence1
0,0,1.000000,Canon EOS REBEL T7i Body EOS Rebel T7i The EOS...
1,1,0.500000,PHILIPS 24 LCD LED BACKLIGHT 1920X1080 VGA HDM...
2,2,1.000000,Ổ Cứng SSD SAMSUNG 970 PRO 512GB NVMe M.2 PCIe...
3,3,0.400000,Intel Core i9-9900X 3.5 GHz Ten-Core LGA 2066 ...
4,4,0.250000,Canon CL-38 Inkjet Cartridge Page Life 205pp 9...
...,...,...,...
1405,1405,1.000000,7.5 cu. ft. Smart Electric Dryer with Steam Sa...
1406,1406,1.000000,Kendal Slate Ambleside Beige KS01 Eastern CT F...
1407,1407,1.000000,Profiles Matte Cappuccino 0009 2 T and H Floor...
1408,1408,0.977778,5.9 cu. ft. Freestanding Electric Range with C...


In [60]:
pd.set_option("max_colwidth", 100)
pd.set_option('display.max_rows', None)

In [69]:
len(df_join_sentence[df_join_sentence.f1==0.000000])

101

In [62]:
real[real.label==2]

,sentence1,label
2439,Ổ Cứng SSD SAMSUNG 970 PRO 512GB NVMe M.2 PCIe Gen 3 x4 512MB Cache (MZ-V7P512BW) Nâng cao hiệu ...,2
4671,"Samsung 970 PRO 512GB MZ-V7P512BW Disco Solido SSD, M.2 2280, PCIe 3.0 x4 NVMe, 512GB, controlad...",2
5378,Samsung 512GB 970 Pro NVMe M.2 MZ-V7P512BW Kapasite: 512GBOkuma/Yazma: 3500MB/s / 2300MB/sArabir...,2
6647,Samsung 970 PRO 512GB - NVMe PCIe M.2 2280 SSD | MZ-V7P512BW Experience the best. For intensive ...,2
9484,Samsung 970 PRO 512 Go SSD M.2 - PCI-Express 3.0 NVMe - Contrôleur Samsung Phoenix - Lecture max...,2


In [74]:
pd.merge(real, df_f1, how="left", on=["label", "label"]).sort_values(by=['label']).to_excel('f1_per_cluster_baseline.xlsx')